In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\2\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\2\test'
checkpoint_path = r'C:\Users\24306\Desktop\RegNetY008\2\RegNetY008.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[84.0, 7.0, 13.0, 37.0, 47.0, 96.0, 66.0, 52.0, 95.0, 4.0, 13.0, 16.0, 37.0, 7.0, 52.0, 37.0, 32.0, 95.0, 23.0, 13.0, 16.0, 47.0, 30.0, 86.0, 45.0, 66.0, 47.0, 55.0, 100.0, 93.0, 84.0, 96.0, 84.0, 37.0, 86.0, 32.0, 16.0, 100.0, 10.0, 32.0, 95.0, 7.0, 47.0, 4.0, 93.0, 66.0, 4.0, 80.0, 86.0, 23.0, 13.0, 86.0, 13.0, 55.0, 23.0, 95.0, 40.0, 40.0, 86.0, 7.0, 45.0, 23.0, 30.0, 30.0, 20.0, 37.0, 32.0, 16.0, 16.0, 86.0, 80.0, 45.0, 84.0, 80.0, 37.0, 86.0, 37.0, 13.0, 4.0, 52.0, 45.0, 95.0, 35.0, 95.0, 37.0, 13.0, 100.0, 32.0, 80.0, 100.0, 35.0, 93.0, 30.0, 47.0, 35.0, 100.0, 93.0, 100.0, 10.0, 37.0, 47.0, 93.0, 84.0, 20.0, 96.0, 86.0, 47.0, 20.0, 96.0, 23.0, 80.0, 96.0, 30.0, 35.0, 7.0, 86.0, 7.0, 23.0, 30.0, 37.0, 4.0, 37.0, 39.0, 13.0, 20.0, 96.0, 32.0, 86.0, 84.0, 95.0, 20.0, 55.0, 37.0, 37.0, 86.0, 55.0, 32.0, 37.0, 7.0, 95.0, 52.0, 16.0, 37.0, 95.0, 52.0, 80.0, 30.0, 7.0, 95.0, 86.0, 39.0, 37.0, 47.0, 47.0, 66.0, 45.0, 39.0, 47.0, 47.0, 45.0, 93.0, 100.0, 20.0, 16.0, 100.0, 96.0, 45.0, 45

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
RegNetY008 =tf.keras.applications.regnet.RegNetY008(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
RegNetY008.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = RegNetY008(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 regnety008 (Functional)     (None, 7, 7, 768)         5524056   
                                                                 
 global_average_pooling2d (G  (None, 768)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 769       
                                                                 
Total params: 5,524,825
Trainable params: 5,494,937
Non-trainable params: 29,888
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                    callbacks=[cp_callback])
#要改
np.savetxt('RegNetY008_loss_2.txt',history.history['loss'])
#要改
np.savetxt('RegNetY008_valloss_2.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Mon Apr 17 09:51:04 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 464.5527

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\2\RegNetY008.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\2\RegNetY008.ckpt\assets


591/591 [==============================] - 78s 112ms/step - loss: 464.5527 - val_loss: 7.4041
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 16.6854

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\2\RegNetY008.ckpt\assets


...

591/591 [==============================] - 112s 190ms/step - loss: 0.1162 - val_loss: 0.0802
Epoch 177/300
591/591 [==============================] - 91s 154ms/step - loss: 0.1640 - val_loss: 0.3162
Epoch 178/300
591/591 [==============================] - 91s 154ms/step - loss: 0.0786 - val_loss: 0.3419
Epoch 179/300
591/591 [==============================] - 90s 153ms/step - loss: 0.1116 - val_loss: 0.2583
Epoch 180/300
591/591 [==============================] - 91s 154ms/step - loss: 0.0957 - val_loss: 0.3500
Epoch 181/300
591/591 [==============================] - ETA: 0s - loss: 0.1647

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\2\RegNetY008.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\RegNetY008\2\RegNetY008.ckpt\assets


591/591 [==============================] - 111s 188ms/step - loss: 0.1647 - val_loss: 0.0715
Epoch 182/300
591/591 [==============================] - 93s 157ms/step - loss: 0.1296 - val_loss: 0.7265
Epoch 183/300
591/591 [==============================] - 92s 156ms/step - loss: 0.0962 - val_loss: 0.1534
Epoch 184/300
591/591 [==============================] - 92s 155ms/step - loss: 0.1386 - val_loss: 0.2748
Epoch 185/300
591/591 [==============================] - 93s 157ms/step - loss: 0.1110 - val_loss: 0.1506
Epoch 186/300
591/591 [==============================] - 91s 154ms/step - loss: 0.0972 - val_loss: 0.6515
Epoch 187/300
591/591 [==============================] - 91s 155ms/step - loss: 0.0741 - val_loss: 0.6224
Epoch 188/300
591/591 [==============================] - 92s 155ms/step - loss: 0.1003 - val_loss: 0.3195
Epoch 189/300
591/591 [==============================] - 92s 156ms/step - loss: 0.0933 - val_loss: 0.2045
Epoch 190/300
591/591 [==============================] - 92

591/591 [==============================] - 126s 213ms/step - loss: 0.1171 - val_loss: 0.1234
Epoch 259/300
591/591 [==============================] - 126s 213ms/step - loss: 0.0836 - val_loss: 0.1844
Epoch 260/300
591/591 [==============================] - 126s 214ms/step - loss: 0.0673 - val_loss: 0.1961
Epoch 261/300
591/591 [==============================] - 126s 214ms/step - loss: 0.0907 - val_loss: 0.2103
Epoch 262/300
591/591 [==============================] - 119s 201ms/step - loss: 0.0692 - val_loss: 0.1383
Epoch 263/300
591/591 [==============================] - 127s 214ms/step - loss: 0.0808 - val_loss: 0.3540
Epoch 264/300
591/591 [==============================] - 124s 210ms/step - loss: 0.1358 - val_loss: 0.1569
Epoch 265/300
591/591 [==============================] - 125s 212ms/step - loss: 0.0784 - val_loss: 0.9320
Epoch 266/300
591/591 [==============================] - 125s 212ms/step - loss: 0.0887 - val_loss: 0.3010
Epoch 267/300
591/591 [============================